In [3]:
"""
DIABETES PREDICTION USING WIDE DEEP MODEL
Dataset: Diabetes from Kaggle
================================================
Pham Ngoc Hung hungpn@soict.hust.edu.vn
Aug 1st 2018
"""

import time
start_time = time.time()
import os
import glob
import csv
import math
import numpy as np 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.autograd import Variable
from torch.utils.data import DataLoader

import pandas as pd 
from collections import namedtuple

from sklearn import metrics
from sklearn import preprocessing

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler


from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt 

use_cuda = torch.cuda.is_available()

In [4]:
class WideDeepLoader(Dataset):
    """Helper to facilitate loading the data to the pytorch models.
    Parameters:
    --------
    data: namedtuple with 3 elements - (wide_input_data, deep_inp_data, target)
    """
    def __init__(self, data):

        self.X_wide = data.wide
        self.X_deep = data.deep
        self.Y = data.labels

    def __getitem__(self, idx):

        xw = self.X_wide[idx]
        xd = self.X_deep[idx]
        y  = self.Y[idx]

        return xw, xd, y

    def __len__(self):
        return len(self.Y)

In [5]:
class WideDeep(nn.Module):
	"""
	Parameters:
	-------------
	wide_dim (int): wide dimensions
	hidden_layers (list): list with the number of units per hidden layer
	n_class (int): number of classes. Defaults to 1 if logistic or regression
	"""
	def __init__(self, wide_dim, deep_cols_idx, embeddings_input, hidden_layers, dropout, n_class):
		super(WideDeep, self).__init__()
		self.wide_dim = wide_dim
		self.deep_cols_idx = deep_cols_idx
		self.hidden_layers = hidden_layers
		self.n_class = n_class
		self.dropout = dropout
		self.embeddings_input = embeddings_input
		# Build the embedding layers to be passed through the deep-side
		#Co the viet tong quat cho cac embedding layer
		#for embname,emb_idx,emb_dim in self.embeddings_input:
        #    setattr(self, 'emb_layer_'+col, nn.Linear(len(emb_idx, emb_dim))
        
		emb_diagnosis = self.embeddings_input[0]
		emb_medication = self.embeddings_input[1]
		emb_labtest = self.embeddings_input[2]

		emb_diagnosis_idx = emb_diagnosis[1]
		emb_diagnosis_dim = emb_diagnosis[2]

		emb_medication_idx = emb_medication[1]
		emb_medication_dim = emb_medication[2]

		emb_labtest_idx = emb_labtest[1]
		emb_labtest_dim = emb_labtest[2]

		# Dung nn.Linear de lam mot embedding layer (Khong dung duoc nn.Embedding vi dau vao la binary vector)	
		self.emb_diagnosis_layer = nn.Linear(len(emb_diagnosis_idx), emb_diagnosis_dim)
		self.emb_medication_layer = nn.Linear(len(emb_medication_idx), emb_medication_dim)
		self.emb_labtest_layer = nn.Linear(len(emb_labtest_idx), emb_labtest_dim)

		n_input_layer1 = len(deep_cols_idx) + emb_diagnosis_dim + emb_medication_dim + emb_labtest_dim
		#input layer
		self.linear_1 = nn.Linear(n_input_layer1, self.hidden_layers[0])
		if self.dropout:
			self.linear_1_drop = nn.Dropout(self.dropout[0])

		# Cac hidden layers
		for i, h in enumerate(self.hidden_layers[1:],1):
			setattr(self, 'linear_' + str(i+1), nn.Linear(self.hidden_layers[i-1], self.hidden_layers[i]))
			if self.dropout:
				setattr(self, 'linear_'+str(i+1)+'_drop', nn.Dropout(self.dropout[i]))
		
		#Them 1 tang linear truoc khi output
		n_linear_output = 512
		self.linear_out = nn.Linear(self.hidden_layers[-1]+self.wide_dim, n_linear_output)
		
		# output 
		self.output = nn.Linear(n_linear_output, n_class)
		#self.output = nn.Linear(self.hidden_layers[-1]+self.wide_dim, n_class)  

	def compile(self, method="logistic", optimizer="Adam", learning_rate=0.001, momentum=0.0):
		"""Warapper to set the activation, loss and the optimizer
		Parameters:
		-------------
		method (str): regression, logistic, or multiclass
		optimizer (str): SGD, Adam, RMSprop
		"""
		if method == 'regression':
			self.activation , self.criterion = None, F.mse_loss
		if method == 'logistic':
			self.activation, self.criterion = F.sigmoid, F.binary_cross_entropy
		if method == 'multiclass':
			self.activation, self.criterion = F.softmax, F.cross_entropy 

		if optimizer == "Adam":
			self.optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
		if optimizer == "RMSprop":
			self.optimizer = torch.optim.RMSprop(self.parameters(), lr=learning_rate)
		if optimizer == "SGD":
			self.optimizer = torch.optim.SGD(self.parameters(), lr=learning_rate, momentum=momentum)	

		self.method = method

	def forward(self, X_wide, X_deep):

		#Lay cac embedding	
		emb_diagnosis = self.embeddings_input[0]
		emb_medication = self.embeddings_input[1]
		emb_labtest = self.embeddings_input[2]

		emb_diagnosis_idx = emb_diagnosis[1]
		emb_diagnosis_dim = emb_diagnosis[2]

		emb_medication_idx = emb_medication[1]
		emb_medication_dim = emb_medication[2]

		emb_labtest_idx = emb_labtest[1]
		emb_labtest_dim = emb_labtest[2]

		deep_inp_emb_diagnosis = torch.cat([X_deep[:, emb_diagnosis_idx].float()],1)
		deep_inp_emb_medication = torch.cat([X_deep[:, emb_medication_idx].float()],1)
		deep_inp_emb_labtest = torch.cat([X_deep[:, emb_labtest_idx].float()],1)

		#x_deep_inp1 = [X_deep[:,self.deep_cols_idx].float()]
		x_deep_emb1 = F.relu(self.emb_diagnosis_layer(deep_inp_emb_diagnosis))
		x_deep_emb2 = F.relu(self.emb_medication_layer(deep_inp_emb_medication))
		x_deep_emb3 = F.relu(self.emb_labtest_layer(deep_inp_emb_labtest))

		deep_inp_all = torch.cat([X_deep[:,self.deep_cols_idx].float(), x_deep_emb1, x_deep_emb2, x_deep_emb3], 1)
		#print(deep_in_all)

		x_deep = F.relu(self.linear_1(deep_inp_all)) # forward qua input layer dau tien
		if self.dropout:
			x_deep = self.linear_1_drop(x_deep)
		for i in range(1, len(self.hidden_layers)):
			x_deep = F.relu(getattr(self, 'linear_'+str(i+1))(x_deep))  #forward qua cac hidden layers tiep theo
			if self.dropout:
				x_deep = getattr(self, 'linear_'+str(i+1)+'_drop')(x_deep)

		#Deep + Wide sides
		#wide_deep_input = torch.cat([x_deep, X_wide[:,0:53].float(), X_wide[:,2090:2980].float()], 1) #gop cac phan x_deep layer cuoi cung voi X_wide de duoc input cho layer cuoi cung
		wide_deep_input = torch.cat([x_deep, X_wide.float()], 1)
		n_linear_output = 512
		x_wd_out = F.relu(self.linear_out(wide_deep_input))
		out = F.sigmoid(self.output(x_wd_out))
		#out = F.sigmoid(self.output(wide_deep_input))  #qua output layer
		return out

	def predict(self, dataset):
		X_w = Variable(torch.from_numpy(dataset.wide)).float()
		X_d = Variable(torch.from_numpy(dataset.deep))

		if use_cuda:
			X_w, X_d = X_w.cuda(), X_d.cuda()

		net = self.eval()
		if use_cuda:
			net = net.cuda()
		#pred_out = net(X_w,X_d).cpu()
		pred_out = net(X_w,X_d)
		pred = (pred_out > 0.5).squeeze(1).data.cpu().numpy()
		
		#return (pred > 0.5).squeeze(1).data.numpy()
		return pred

	def predict_proba(self, dataset):
		"""Predict probability for data set
		"""
		X_w = Variable(torch.from_numpy(dataset.wide)).float()
		X_d = Variable(torch.from_numpy(dataset.deep))
		if use_cuda:
			X_w, X_d = X_w.cuda(), X_d.cuda()

		# set the model in evaluation mode so dropout is not applied
		net = self.eval()
		if use_cuda:
			net = net.cuda()
		#pred = net(X_w,X_d).cpu()
		pred = net(X_w, X_d)
		pred = pred.squeeze(1).data.cpu().numpy()
		probs = np.zeros([pred.shape[0],2])
		probs[:,0] = 1-pred
		probs[:,1] = pred
		#return probs
		return pred	

	def train_validate(self, train_dataset, validate_dataset, fold, n_epochs, batch_size):
		
		widedeep_dataset = WideDeepLoader(train_dataset)
		train_loader = torch.utils.data.DataLoader(dataset=widedeep_dataset, batch_size=batch_size, shuffle=True)

		net = self.train()
		epoch_count = 0
		#global auc_max
		auc_max = 0 #Do auc voi moi fold tim auc_max
		best_epoch = 0 
		accuracy_score_save = 0
		for epoch in range(n_epochs):
			total = 0
			correct = 0
			for i, (X_wide, X_deep, target) in enumerate(train_loader):
				#print("X_wide: \n", X_wide)
				#print("X_deep: \n", X_deep)
				#print("target: \n", target)				
				X_w = Variable(X_wide)
				X_d = Variable(X_deep)
				y = Variable(target).float()
				if use_cuda:
					X_w, X_d, y = X_w.cuda(), X_d.cuda(), y.cuda()
					net = net.cuda()

				self.optimizer.zero_grad()
				y_pred = net(X_w, X_d)
				loss = F.binary_cross_entropy(y_pred, y)
				loss.backward()
				self.optimizer.step()

				total +=y.size(0)
				y_pred_cat	= (y_pred > 0.5).squeeze(1).float()
				correct+=float((y_pred_cat == y).sum().item())

			print ('Epoch {} of {}, Loss: {}, Train accuracy: {}'.format(epoch+1, n_epochs, round(loss.item(),6), round(correct/total,6)))
			y_val_pred = self.predict(validate_dataset)
			y_val_true = validate_dataset.labels
			accuracy_score = metrics.accuracy_score(y_val_true, y_val_pred)
			print("Accuracy score (Cross-Validation) = ", accuracy_score)
			y_val_prob = self.predict_proba(validate_dataset)
			auc = metrics.roc_auc_score(y_val_true, y_val_prob)

			print("ROC AUC score (Cross-Validation) = ", auc)
			if(auc > auc_max):
				epoch_count = 0
				auc_max = auc
				best_epoch = epoch  #Luu epoch hien tai
				best_auc = accuracy_score
				torch.save(self.state_dict(), os.path.join(MODEL_DIR, "current_model_saved.pfl")) #Luu model hien tai voi cac parameters
			else:
				epoch_count = epoch_count + 1  #Do auc khong tang thi khong luu
			if epoch_count >= 50: #Neu qua 50 epoch ma auc khong tang thi dung
				break
		#Load lai model da save
		self.load_state_dict(torch.load(MODEL_DIR+"/current_model_saved.pfl"))  #load again current_model_save for next training
		#Luu lai model tot nhat ung voi fold validation hien tai
		model_fn = "wide_deep_model_{}_{}.pfl".format(fold, best_epoch)
		torch.save(self.state_dict(), os.path.join(MODEL_DIR, model_fn))
		print(model_fn, "saved")

		with open(traininglog_fn, mode='a') as logfile:
			logfile = csv.writer(logfile, delimiter=',')
			logfile.writerow([model_fn, best_auc, auc_max])

In [6]:
def training():
    
    with open(traininglog_fn, mode='w') as logfile:
        logfile = csv.writer(logfile, delimiter=',')
        logfile.writerow(['model_fn', 'Validate Accuracy score', 'Validate AUC score'])
        
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1981)
    sm = SMOTE(random_state=4, ratio=0.3)
    # 10 fold
    n_epochs = 2000
    batch_size = 32
    fold = 0
    #Split TRAIN_VAL dataset into 5-fold, 4 for training and 1 for cross validation
    for train_index, val_index in skf.split(X_TRAIN_VAL, Y_TRAIN_VAL):
        X_train, X_val = X_TRAIN_VAL[train_index], X_TRAIN_VAL[val_index]      #get train and crossvalidation data by splitted indexes
        y_train, y_val = Y_TRAIN_VAL[train_index], Y_TRAIN_VAL[val_index]
        #print("X_train indices: ", train_index)
        #print("X_val indices: ", val_index)
        print("y_train len: ", len(y_train), "y_val len:", len(y_val))
        unique, count = np.unique(y_val, return_counts = True)
        print("y_val values: ", unique, "count: ", count)
        # oversample tap train sau khi da split train-val (tranh oversample ca validate dataset)
        #print("BEFORE: y_train_values: ", np.bincount(y_train))
        #X_train, y_train = sm.fit_sample(X_train, y_train.ravel())
        #print("AFTER SMOTE: y_train len = ", len(y_train))
        #unique, count = np.unique(y_train, return_counts = True)
        #print("y_train values: ", unique, "count: ", count)
        X_train_wide = X_train       
        X_train_deep = X_train
        X_val_wide = X_val
        X_val_deep = X_val
        #print("X_train_deep: ", X_train_deep)
        #wrap to train_dataset(wide, deep, labels)
        train_dataset = namedtuple('train_dataset', 'wide, deep, labels')
        train_dataset = train_dataset(X_train_wide, X_train_deep, y_train)
        #wrap to validate_dataset(wide, deep, labels)
        validate_dataset = namedtuple('validate_dataset', 'wide, deep, labels')
        validate_dataset = validate_dataset(X_val_wide, X_val_deep, y_val)
        print("Train-Validate fold ", fold)

        wd_model = WideDeep(wide_dim, deep_cols_idx, embeddings_input, hidden_layers, dropout, n_class)
        print(wd_model)
        wd_model.compile(method='logistic', optimizer='SGD') 
        wd_model.train_validate(train_dataset, validate_dataset, fold, n_epochs, batch_size)
        fold += 1

In [15]:
def testing():
	#test_dataset
	X_test_wide = X_TEST #All columns go to wide part
	X_test_deep = X_TEST
	test_dataset = namedtuple('test_dataset', 'wide, deep, labels')
	test_dataset = test_dataset(X_test_wide, X_test_deep, Y_TEST)

	with open(testresult_fn, mode='w') as outfile:
		outfile = csv.writer(outfile, delimiter=',')
		outfile.writerow(['model_fn', 'Accuracy score', 'AUC score', 'Sensitivity', 'Specificity'])

	list_model_fn = sorted(glob.glob(MODEL_DIR+"/*.pfl"))
	wd_model = WideDeep(wide_dim, deep_cols_idx, embeddings_input, hidden_layers, dropout, n_class)
	print(wd_model)
	wd_model.compile(method='logistic', optimizer='SGD') 
	#y_prob_mtx = range(len(Y_TEST))
	y_prob_mtx = []
	print("==================================TESTING RESULT=======================================")
	y_true = test_dataset.labels
	for model_fn in list_model_fn:
		print(model_fn)
		wd_model.load_state_dict(torch.load(model_fn))
		y_predict = wd_model.predict(test_dataset)
		y_prob = wd_model.predict_proba(test_dataset)
		#y_prob_mtx = np.vstack((y_prob_mtx, y_prob))
		y_prob_mtx.append(y_prob)
		print("y_true:\n", y_true)
		#print(len(y_true))
		print("y_predict:\n", y_predict)
		print("y_prob: \n", y_prob)
		accuracy_score = metrics.accuracy_score(y_true, y_predict)
		auc_score = metrics.roc_auc_score(y_true, y_prob)
		cm = metrics.confusion_matrix(y_true, y_predict)
		sensitivity = cm[0,0]/(cm[0,0] + cm[0,1])    # sensitivity = TP/(TP+FN)
		specificity = cm[1,1]/(cm[1,1] + cm[1,0])    # specificity = TN/(TN+FP)
		print("Accuracy score (Testing Set) = ", accuracy_score)
		print("ROC AUC score  (Testing Set) = ", auc_score)
		print("Sensitivity    (Testing Set) = ", sensitivity)
		print("Specificity    (Testing Set) = ", specificity)
		print("=========================================================================================")
		with open(testresult_fn, mode='a') as outfile:
			outfile = csv.writer(outfile, delimiter=',')
			outfile.writerow([model_fn, accuracy_score, auc_score, sensitivity, specificity])
	
	print("ENSEMBLE 5 MODELS")
	y_prob_mtx = np.array(y_prob_mtx)
	print("y_prob_mtx: ", y_prob_mtx.shape, "\n", y_prob_mtx)
	y_prob_ensemble = [sum(y_prob_mtx[1:,col])/5 for col in range(np.size(y_prob_mtx, 1))] #model_fn dau tien la luu trung gian
	y_pred_ensemble = [np.float(each > 0.5) for each in y_prob_ensemble]
	auc_score_ensemble = metrics.roc_auc_score(y_true, y_prob_ensemble)
	accuracy_score_ensemble = metrics.accuracy_score(y_true, y_pred_ensemble)
	cm = metrics.confusion_matrix(y_true, y_pred_ensemble)
	sensitivity_ensemble = cm[0,0]/(cm[0,0] + cm[0,1])
	specificity_ensemble = cm[1,1]/(cm[1,1] + cm[1,0])

	print("Accuracy score (Testing Set) = ", accuracy_score_ensemble)
	print("ROC AUC score  (Testing Set) = ", auc_score_ensemble)
	print("Sensitivity    (Testing Set) = ", sensitivity_ensemble)
	print("Specificity    (Testing Set) = ", specificity_ensemble)
	with open(testresult_fn, mode='a') as outfile:
			outfile = csv.writer(outfile, delimiter=',')
			outfile.writerow(["ensemble", accuracy_score_ensemble, auc_score_ensemble, sensitivity_ensemble, specificity_ensemble])
		
	#Ghi log nhung sample doan sai	
	#with open("logfile.csv", mode='w') as logfile:
	#	logfile = csv.writer(logfile, delimiter=',')
	#	logfile.writerow(["test sample", "model0_out", "model1_out", "model2_out", "model3_out", "model4_out", "y_pred", "y_true"])
	#	for i in range(np.size(y_prob_mtx, 1)):
	#		if(y_pred_ensemble[i] != y_true[i]):
	#			logfile.writerow([i, y_prob_mtx[1, i], y_prob_mtx[2, i], y_prob_mtx[3, i], y_prob_mtx[4, i], y_prob_mtx[5, i], y_pred_ensemble[i], y_true[i]])
	
	#Ve ROC
	#print("Thresholds: \n", thresholds)
	#print("False Positive Rate: ", fpr)
	#print("True Positive Rate: ", tpr)
	"""
	plt.figure()
	plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.4f)' %metrics.auc(fpr,tpr))
	plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
	plt.xlim([0.0, 1.0])
	plt.ylim([0.0, 1.05])
	plt.xlabel('False Positive Rate')
	plt.ylabel('True Positive Rate')
	plt.title('Receiver operating characteristic example')
	plt.legend(loc='lower right')
	plt.show()
	"""

In [7]:
###########################################################################################
# MAIN

MODEL_DIR = 'model'
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

traininglog_fn = 'training_log.csv'
testresult_fn = 'test_result.csv'

traininglog_fn = 'training_log.csv'
testresult_fn = 'test_result.csv'

patientds = pd.read_csv("trainingSet/patientdata_1311.csv")
print(patientds.shape)

#continuous_cols = ['Age', 'MeanBMI', 'MaxBMI','MinBMI', 'BMIDiff','meanSystolicBP', 'meanDiastolicBP', 'maxSystolicBP', 'maxDiastolicBP', 'minSystolicBP',	'minDiastolicBP', 'SystDiff', 'DiastDiff']
#patientds[continuous_cols].as_matrix().astype(np.float)

#print(patientds.iloc[0:10, 0:5])
X = np.array(patientds.iloc[:,3:]) #0-index, 1-PatientGuid, 2-DMindicator
Y = np.array(patientds['DMIndicator'])
#print("X: \n", len(X), X)
#print("Y: \n", len(Y), Y)

patientdata = patientds.iloc[:,3:] #Bo di 3 cot dau tien

(9948, 1312)


In [8]:
deep_cols = patientdata.columns[0:53]   #53 cot dau Gender, Age, BMI, .... (cho vao deep part)
#print(deep_cols)
deep_cols_idx = list(range(0,53))
#print(deep_cols_idx)

diagnosis_cols = patientdata.columns[53:204] #151 diagnosis features columns
diagnosis_cols_idx = list(range(53,204))

medication_cols = patientdata.columns[204:338] #134 medication features columns
medication_cols_idx = list(range(204,338))

labtest_cols = patientdata.columns[338:418] #80 labtest features columns
labtest_cols_idx = list(range(338,418))

crossed_cols = patientdata.columns[418:-1] #890 crossed columns
crossed_cols_idx = list(range(418,1309))

emb_diagnosis_dim = 32 	#Dimension of embedding diaggnosis
emb_medication_dim = 16 #Dimension of embedding medication
emb_labtest_dim = 8 	#Dimension of embedding labtest

emb_diagnosis = ('diagnosis', diagnosis_cols_idx, emb_diagnosis_dim) 		# ('diagnosis', diagnosis_cols_idx, emb_diagnosis_dim)
emb_medication = ('medication', medication_cols_idx, emb_medication_dim) 	#('medication', medication_cols_idx, emb_medication_dim)
emb_labtest = ('labtest', labtest_cols_idx, emb_labtest_dim)  				#('lab test', labtest_cols_idx, emb_labtest_dim)

embeddings_input = [emb_diagnosis, emb_medication, emb_labtest]
#print("embeddings_input:", embeddings_input)

wide_dim = patientdata.shape[1] #Tat ca di vao wide side

#Cach 1: Neu chon layer dau ra co so noron bang layer dau vao cho phan deep
#n_out_layer = len(deep_cols_idx) + emb_diagnosis_dim + emb_medication_dim + emb_labtest_dim  
#hidden_layers = [256, n_out_layer] #Dung 2 hidden layers (mang phinh o giua)

#Cach 2: Dung cac lop an thu gon dan
hidden_layers = [256, 128]          
n_class = 1
# Neu dung dropout
#dropout = [0.5, 0.2]

# Neu khong dung dropout
dropout = [] 

In [9]:
# train/test split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=1981)
for train_val_idx, test_idx in sss.split(X, Y):
    print("TRAIN indices: ", train_val_idx)
    print("TEST indices: ", test_idx)
    X_TRAIN_VAL, X_TEST = X[train_val_idx], X[test_idx]
    Y_TRAIN_VAL, Y_TEST = Y[train_val_idx], Y[test_idx]
#scale train and test set 
min_max_scaler = preprocessing.MinMaxScaler()
X_TRAIN_VAL = min_max_scaler.fit_transform(X_TRAIN_VAL)
#print(min_max_scaler.data_max_)
X_TEST = min_max_scaler.transform(X_TEST)
print("X_TRAIN_VAL: \n", X_TRAIN_VAL)
print("X_TEST: \n", X_TEST)

print("X_TRAIN_VAL len / X_TEST len = ", len(X_TRAIN_VAL),"/",len(X_TEST))
unique, count = np.unique(Y_TEST, return_counts = True)
print("Y_TEST values: ", unique, "count: ", count)
print("Positive class percent: ", count[1]/(count[0]+count[1])*100)

TRAIN indices:  [9258 8330 4658 ... 3352 4901 1619]
TEST indices:  [4059 8274 5741 ... 6603 9004 5876]
X_TRAIN_VAL: 
 [[1.         0.79166667 0.56442208 ... 0.         0.         0.        ]
 [1.         0.36111111 0.30654088 ... 0.         0.         0.        ]
 [1.         0.45833333 0.43188599 ... 0.         0.         0.        ]
 ...
 [1.         0.19444444 0.16319577 ... 0.         0.         0.        ]
 [0.         0.19444444 0.20652741 ... 0.         0.         0.        ]
 [0.         0.91666667 0.1728862  ... 0.         0.         0.        ]]
X_TEST: 
 [[0.         0.375      0.20827826 ... 0.         0.         0.        ]
 [1.         0.375      0.19863472 ... 0.         0.         0.        ]
 [0.         0.72222222 0.29810689 ... 0.         0.         0.        ]
 ...
 [1.         0.58333333 0.5254797  ... 0.         0.         0.        ]
 [1.         0.63888889 0.34016124 ... 0.         0.         0.        ]
 [0.         0.08333333 0.38678788 ... 0.         0.      

In [10]:
#TRAINING
training()
print("Time to train model taken (seconds): %d" %(time.time()-start_time))

y_train len:  5570 y_val len: 1393
y_val values:  [0 1] count:  [1126  267]
Train-Validate fold  0
WideDeep(
  (emb_diagnosis_layer): Linear(in_features=151, out_features=32, bias=True)
  (emb_medication_layer): Linear(in_features=134, out_features=16, bias=True)
  (emb_labtest_layer): Linear(in_features=80, out_features=8, bias=True)
  (linear_1): Linear(in_features=109, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=128, bias=True)
  (linear_out): Linear(in_features=1437, out_features=512, bias=True)
  (output): Linear(in_features=512, out_features=1, bias=True)
)


/home/dce/.local/lib/python3.5/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/dce/.local/lib/python3.5/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/dce/.local/lib/python3.5/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch 1 of 2000, Loss: 0.632919, Train accuracy: 0.732136
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) =  0.4092941106033089
Epoch 2 of 2000, Loss: 0.702059, Train accuracy: 0.808618
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) =  0.3829504859600455
Epoch 3 of 2000, Loss: 0.543376, Train accuracy: 0.808618
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) =  0.37473639744280574
Epoch 4 of 2000, Loss: 0.52117, Train accuracy: 0.808618
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) =  0.3724196885332056
Epoch 5 of 2000, Loss: 0.706725, Train accuracy: 0.808618
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) =  0.37364040952361943
Epoch 6 of 2000, Loss: 0.759452, Train accuracy: 0.808618
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) =  0.37

Epoch 50 of 2000, Loss: 0.756407, Train accuracy: 0.808618
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) =  0.7151894944818089
Epoch 51 of 2000, Loss: 0.207481, Train accuracy: 0.808618
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) =  0.7195169004995976
Epoch 52 of 2000, Loss: 0.777237, Train accuracy: 0.808618
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) =  0.7241370134578669
Epoch 53 of 2000, Loss: 0.873664, Train accuracy: 0.808618
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) =  0.7277592618463155
Epoch 54 of 2000, Loss: 0.295556, Train accuracy: 0.808618
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) =  0.7311436858456237
Epoch 55 of 2000, Loss: 1.474533, Train accuracy: 0.808618
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) = 

Epoch 99 of 2000, Loss: 0.132537, Train accuracy: 0.809336
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) =  0.7803367460301621
Epoch 100 of 2000, Loss: 0.288849, Train accuracy: 0.809336
Accuracy score (Cross-Validation) =  0.8083273510409189
ROC AUC score (Cross-Validation) =  0.7806660413382029
Epoch 101 of 2000, Loss: 0.085279, Train accuracy: 0.809695
Accuracy score (Cross-Validation) =  0.8090452261306532
ROC AUC score (Cross-Validation) =  0.7810485560899676
Epoch 102 of 2000, Loss: 0.160853, Train accuracy: 0.809874
Accuracy score (Cross-Validation) =  0.8090452261306532
ROC AUC score (Cross-Validation) =  0.7814111135503357
Epoch 103 of 2000, Loss: 0.502185, Train accuracy: 0.810233
Accuracy score (Cross-Validation) =  0.8090452261306532
ROC AUC score (Cross-Validation) =  0.7817503875040747
Epoch 104 of 2000, Loss: 0.189283, Train accuracy: 0.810592
Accuracy score (Cross-Validation) =  0.8097631012203876
ROC AUC score (Cross-Validatio

...

Epoch 335 of 2000, Loss: 0.539613, Train accuracy: 0.847065
Accuracy score (Cross-Validation) =  0.8225574712643678
ROC AUC score (Cross-Validation) =  0.8284665927696684
Epoch 336 of 2000, Loss: 1.163351, Train accuracy: 0.848142
Accuracy score (Cross-Validation) =  0.8232758620689655
ROC AUC score (Cross-Validation) =  0.8284799476488736
Epoch 337 of 2000, Loss: 0.204175, Train accuracy: 0.847783
Accuracy score (Cross-Validation) =  0.8225574712643678
ROC AUC score (Cross-Validation) =  0.8285200122864889
Epoch 338 of 2000, Loss: 0.158226, Train accuracy: 0.847065
Accuracy score (Cross-Validation) =  0.8225574712643678
ROC AUC score (Cross-Validation) =  0.8285634156439056
Epoch 339 of 2000, Loss: 1.262908, Train accuracy: 0.848142
Accuracy score (Cross-Validation) =  0.8232758620689655
ROC AUC score (Cross-Validation) =  0.8285433833250977
Epoch 340 of 2000, Loss: 1.514272, Train accuracy: 0.848142
Accuracy score (Cross-Validation) =  0.8218390804597702
ROC AUC score (Cross-Validati

Epoch 384 of 2000, Loss: 0.579121, Train accuracy: 0.852091
Accuracy score (Cross-Validation) =  0.8189655172413793
ROC AUC score (Cross-Validation) =  0.8291977724061486
Epoch 385 of 2000, Loss: 0.131686, Train accuracy: 0.851912
Accuracy score (Cross-Validation) =  0.8182471264367817
ROC AUC score (Cross-Validation) =  0.8292378370437639
Epoch 386 of 2000, Loss: 0.439696, Train accuracy: 0.852989
Accuracy score (Cross-Validation) =  0.8182471264367817
ROC AUC score (Cross-Validation) =  0.8292044498457511
Epoch 387 of 2000, Loss: 0.92126, Train accuracy: 0.853527
Accuracy score (Cross-Validation) =  0.8211206896551724
ROC AUC score (Cross-Validation) =  0.8293012727199882
Epoch 388 of 2000, Loss: 0.111834, Train accuracy: 0.852809
Accuracy score (Cross-Validation) =  0.819683908045977
ROC AUC score (Cross-Validation) =  0.8292144660051549
Epoch 389 of 2000, Loss: 0.027433, Train accuracy: 0.852809
Accuracy score (Cross-Validation) =  0.819683908045977
ROC AUC score (Cross-Validation)

Epoch 433 of 2000, Loss: 0.560562, Train accuracy: 0.85604
Accuracy score (Cross-Validation) =  0.8125
ROC AUC score (Cross-Validation) =  0.8296084349417059
Epoch 434 of 2000, Loss: 1.221581, Train accuracy: 0.854963
Accuracy score (Cross-Validation) =  0.8125
ROC AUC score (Cross-Validation) =  0.8296017575021034
Epoch 435 of 2000, Loss: 0.2832, Train accuracy: 0.855861
Accuracy score (Cross-Validation) =  0.8132183908045977
ROC AUC score (Cross-Validation) =  0.8295717090238918
Epoch 436 of 2000, Loss: 0.170839, Train accuracy: 0.85622
Accuracy score (Cross-Validation) =  0.8132183908045977
ROC AUC score (Cross-Validation) =  0.8296151123813086
Epoch 437 of 2000, Loss: 0.034925, Train accuracy: 0.85604
Accuracy score (Cross-Validation) =  0.8132183908045977
ROC AUC score (Cross-Validation) =  0.8296267979006131
Epoch 438 of 2000, Loss: 0.115958, Train accuracy: 0.855502
Accuracy score (Cross-Validation) =  0.8132183908045977
ROC AUC score (Cross-Validation) =  0.8295850639030969
Epo

Epoch 482 of 2000, Loss: 0.634252, Train accuracy: 0.858912
Accuracy score (Cross-Validation) =  0.8139367816091954
ROC AUC score (Cross-Validation) =  0.8296351447001162
Epoch 483 of 2000, Loss: 0.058988, Train accuracy: 0.858553
Accuracy score (Cross-Validation) =  0.8146551724137931
ROC AUC score (Cross-Validation) =  0.82964516085952
Epoch 484 of 2000, Loss: 0.09154, Train accuracy: 0.859989
Accuracy score (Cross-Validation) =  0.8146551724137931
ROC AUC score (Cross-Validation) =  0.8296885642169366
Epoch 485 of 2000, Loss: 0.296631, Train accuracy: 0.85963
Accuracy score (Cross-Validation) =  0.8160919540229885
ROC AUC score (Cross-Validation) =  0.8297152739753468
Epoch 486 of 2000, Loss: 0.353386, Train accuracy: 0.858733
Accuracy score (Cross-Validation) =  0.8139367816091954
ROC AUC score (Cross-Validation) =  0.829741983733757
Epoch 487 of 2000, Loss: 0.195454, Train accuracy: 0.859451
Accuracy score (Cross-Validation) =  0.8146551724137931
ROC AUC score (Cross-Validation) =

Epoch 531 of 2000, Loss: 0.247051, Train accuracy: 0.861425
Accuracy score (Cross-Validation) =  0.8168103448275862
ROC AUC score (Cross-Validation) =  0.8297286288545521
Epoch 532 of 2000, Loss: 0.20491, Train accuracy: 0.861066
Accuracy score (Cross-Validation) =  0.8160919540229885
ROC AUC score (Cross-Validation) =  0.829681886777334
Epoch 533 of 2000, Loss: 1.077521, Train accuracy: 0.860707
Accuracy score (Cross-Validation) =  0.8160919540229885
ROC AUC score (Cross-Validation) =  0.8297586773327635
Epoch 534 of 2000, Loss: 0.159941, Train accuracy: 0.862143
Accuracy score (Cross-Validation) =  0.8182471264367817
ROC AUC score (Cross-Validation) =  0.829751999893161
Epoch 535 of 2000, Loss: 0.494192, Train accuracy: 0.861784
Accuracy score (Cross-Validation) =  0.8182471264367817
ROC AUC score (Cross-Validation) =  0.8297353062941545
Epoch 536 of 2000, Loss: 0.070645, Train accuracy: 0.861246
Accuracy score (Cross-Validation) =  0.8182471264367817
ROC AUC score (Cross-Validation)

In [16]:
#TESTING
testing()

WideDeep(
  (emb_diagnosis_layer): Linear(in_features=151, out_features=32, bias=True)
  (emb_medication_layer): Linear(in_features=134, out_features=16, bias=True)
  (emb_labtest_layer): Linear(in_features=80, out_features=8, bias=True)
  (linear_1): Linear(in_features=109, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=128, bias=True)
  (linear_out): Linear(in_features=1437, out_features=512, bias=True)
  (output): Linear(in_features=512, out_features=1, bias=True)
)
==================================TESTING RESULT=======================================
model/current_model_saved.pfl
y_true:
 [0 0 0 ... 0 1 0]
y_predict:
 [0 0 0 ... 1 0 0]
y_prob: 
 [0.01790418 0.04420414 0.13249965 ... 0.60249126 0.3549003  0.02819207]
Accuracy score (Testing Set) =  0.8291457286432161
ROC AUC score  (Testing Set) =  0.8393369384950892
Sensitivity    (Testing Set) =  0.951946975973488
Specificity    (Testing Set) =  0.30998248686514884
model/wide_deep_model_0_494.pfl


/home/dce/.local/lib/python3.5/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


y_true:
 [0 0 0 ... 0 1 0]
y_predict:
 [0 0 0 ... 0 0 0]
y_prob: 
 [0.01603735 0.03655684 0.08040255 ... 0.49368474 0.33832097 0.03620747]
Accuracy score (Testing Set) =  0.830820770519263
ROC AUC score  (Testing Set) =  0.8335040634245362
Sensitivity    (Testing Set) =  0.9507042253521126
Specificity    (Testing Set) =  0.3239929947460595
model/wide_deep_model_4_498.pfl
y_true:
 [0 0 0 ... 0 1 0]
y_predict:
 [0 0 0 ... 1 0 0]
y_prob: 
 [0.01790418 0.04420414 0.13249965 ... 0.60249126 0.3549003  0.02819207]
Accuracy score (Testing Set) =  0.8291457286432161
ROC AUC score  (Testing Set) =  0.8393369384950892
Sensitivity    (Testing Set) =  0.951946975973488
Specificity    (Testing Set) =  0.30998248686514884
ENSEMBLE 5 MODELS
y_prob_mtx:  (6, 2985) 
 [[0.01790418 0.04420414 0.13249965 ... 0.60249126 0.3549003  0.02819207]
 [0.01588133 0.04178319 0.09756716 ... 0.5044864  0.31871545 0.0237464 ]
 [0.01506633 0.03635989 0.0981283  ... 0.5368075  0.32209307 0.016356  ]
 [0.01558185 0.037942